In [1]:
import torch

print(torch.__version__)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

2.0.1+cu117
cuda:0


In [ ]:
from transformers import BertForSequenceClassification, BertTokenizerFast, BertJapaneseTokenizer, Trainer, TrainingArguments
from datasets import load_dataset

base_model_name = 'cl-tohoku/bert-base-japanese-v3'
# base_model_name = 'cl-tohoku/bert-large-japanese-v2'
prefix = base_model_name.split('/')[-1]

id2label = {0: "NEGATIVE", 1: "POSITIVE", 2: "NEUTRAL"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1, "NEUTRAL": 2}

model = BertForSequenceClassification.from_pretrained(
    base_model_name, num_labels=3, id2label=id2label, label2id=label2id)
model = model.to(device)
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-large-japanese-v2')

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average='weighted', zero_division=0)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
from datasets import load_dataset

dataset = load_dataset('dataset_loader.py', name='sentiment_dataset')

In [ ]:
def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True)

train_dataset, test_dataset = dataset['train'].map(tokenize, batched=True), dataset['test'].map(tokenize, batched=True)
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

In [ ]:
train_dataset, test_dataset

In [ ]:
# Poetryが入っていないとログが出力されないので注意
# !pip install tensorboard/ poetry add tensorboard

# トレーニングの設定
training_args = TrainingArguments(
    output_dir='./results',             # 出力フォルダ
    logging_dir='./logs',               # ログ保存フォルダ
    num_train_epochs=50,               # エポック数
    per_device_train_batch_size=1,      # 訓練のバッチサイズ (GPU数によって変える) 8, 1
    per_device_eval_batch_size=4,      # 評価のバッチサイズ (GPU数によって変える) 16 ,4
    gradient_accumulation_steps=2,      # accumulate gradients over 2 batches (GPU数によって変える)
    warmup_steps=500,                   # 学習率スケジューラのウォームアップステップ数
    weight_decay=0.01,                  # 重み減衰の強さ
    save_steps=1000,
    evaluation_strategy="steps",
    eval_steps=100,
    logging_steps=100,
    prediction_loss_only=True,
)

training_args.output_dir = f'./results_{prefix}_{training_args.num_train_epochs}_v1' # 出力フォルダ

# トレーナーの初期化とトレーニング開始
trainer = Trainer(
    model=model,                        # モデル
    args=training_args,                 # 訓練引数
    train_dataset=train_dataset,        # 訓練データセット
    eval_dataset=test_dataset,          # 評価データセット
    compute_metrics=compute_metrics
)

# チェックポイントから学習を再開したいとき
# trainer.train(ignore_keys_for_eval=['last_hidden_state', 'hidden_states', 'attentions'],
            #   resume_from_checkpoint=True)

trainer.train()

In [ ]:
trainer.evaluate(test_dataset)

In [ ]:
trainer.save_state()
trainer.save_model()

In [ ]:
training_args.output_dir